  # This notebook splits the data into training, validation and test sets. Also pre-processes the data
  Data is split by instance, for each problem set 6 instances are used for training, 2 for validation, and 1 for testing

In [3]:
import pandas as pd

In [2]:
column_names = [
    'iterations',
    'instance_type',
    'tw_spread',
    'operator_selection_mechanism',
    'number_of_vertices_to_remove',
    "delta_last_improv", #change to rel_delta... for future work when i want to test on my benchmarks
    "acceptance_ratio",
    "i_last_improv",
    'prev_remove_operator',
    'prev_insert_operator',
    'route_imbalance',
    'capacity_utilization',
    'success_r_op_1',
    'success_r_op_2',
    'success_r_op_3',
    'success_r_op_4',
    'success_r_op_5',
    'success_i_op_1',
    'success_i_op_2',
    'success_i_op_3',
    'delta_cost',
    'new_cost',
    'chosen_remove_operator',
    'chosen_insert_operator'
]


log_df = pd.read_csv("../../results_and_logs/training_alns_iterations_1.log", sep=',', header=None, names=column_names)

In [5]:
log_df['iterations'] = log_df['iterations'].apply(lambda x: int(str(x).split(":")[-1]))

In [12]:
log_df['current_cost'] = log_df['new_cost'] - log_df['delta_cost']

In [13]:
#relativise delta last improv:
log_df['rel_delta_last_improv'] = log_df['delta_last_improv'] / log_df['current_cost']

In [7]:
log_df[['delta_last_improv', 'current_cost', 'rel_delta_last_improv']].head()

,delta_last_improv,current_cost,rel_delta_last_improv
0,0.000000,10267.032510,0.000000
1,-718.948672,9548.083839,-0.075298
2,-91.810910,9456.272928,-0.009709
3,-160.354919,9295.918009,-0.017250
4,-160.354919,9295.918009,-0.017250


In [64]:
def compute_short_long_term_improvement(df, alpha, k):
    improvements = []

    for i in range(len(df)):
        current_cost = df.loc[i, 'current_cost']
        immediate = df.loc[i, 'delta_cost']
        future_window = df['delta_cost'].iloc[i+1:i+1+k]

        # Best improvement (most negative value) in the next k steps
        best_future_improvement = future_window.min() if not future_window.empty else 0

        # Weighted combination
        rel_immediate = immediate / current_cost #added
        rel_best_future_improvement = best_future_improvement / current_cost #added
        combined = alpha * rel_immediate + (1 - alpha) * rel_best_future_improvement
        improvements.append(combined)

    df['short_long_improvement'] = improvements
    return df

In [65]:
df = compute_short_long_term_improvement(log_df, alpha=0.5, k=10)

In [12]:
df.count()
# all columns are complete except for pre_remove_operator and prev_insert_operator since the first iteration in every alns run has no previous values

iterations                      216000
instance_type                   216000
tw_spread                       216000
operator_selection_mechanism    216000
number_of_vertices_to_remove    216000
delta_last_improv               216000
acceptance_ratio                216000
i_last_improv                   216000
prev_remove_operator            215892
prev_insert_operator            215892
route_imbalance                 216000
capacity_utilization            216000
success_r_op_1                  216000
success_r_op_2                  216000
success_r_op_3                  216000
success_r_op_4                  216000
success_r_op_5                  216000
success_i_op_1                  216000
success_i_op_2                  216000
success_i_op_3                  216000
delta_cost                      216000
new_cost                        216000
chosen_remove_operator          216000
chosen_insert_operator          216000
current_cost                    216000
rel_delta_last_improv    

In [70]:
df.shape

(216000, 27)

## Splitting in train, validation and test sets

In [71]:
problem_types = 6
problem_instances = 9
runs_per_instance = 2
iterations_per_run = 2000

In [72]:
# 1. divide dataframe by problem types
training_set, validation_set, test_set = [], [], []
problem_size = problem_instances * runs_per_instance * iterations_per_run
for start in range(0, df.shape[0], problem_size):
    print(" / problem type start: ", start)
    df_by_problem_type = df.iloc[start:start + problem_size]
    # divide in two, first part was done using roulette-wheel, second part was done using random selector
    df_roulette_wheel = df_by_problem_type.iloc[0:18000]
    df_random = df_by_problem_type.iloc[18000:]
    # from each problem type take three instances - two for validation and one for testing
    # training sets:
    df_roulette_wheel_train = df_roulette_wheel.iloc[0:2000*6]
    df_random_train = df_random.iloc[0:2000*6]
    # validation sets:
    df_roulette_wheel_val = df_roulette_wheel.iloc[2000*6:2000*8]
    df_random_val = df_random.iloc[2000*6:2000*8]
    # testing sets:
    df_roulette_wheel_test = df_roulette_wheel.iloc[2000*8:]
    df_random_test = df_random.iloc[2000*8:]

    print("train length of both operator selectors: ", df_roulette_wheel_train.shape[0], df_random_train.shape[0])
    print("validation length of both operator selectors: ", df_roulette_wheel_val.shape[0], df_random_val.shape[0])
    print("test length of both operator selectors: ", df_roulette_wheel_test.shape[0], df_random_test.shape[0])

    training_subset = pd.concat([df_roulette_wheel_train, df_random_train])
    training_set.append(training_subset)

    validation_subset = pd.concat([df_roulette_wheel_val, df_random_val])
    validation_set.append(validation_subset)

    test_subset = pd.concat([df_roulette_wheel_test, df_random_test])
    training_set.append(test_subset) #todo: i changed to the training subset so it has more data to train with

    # process_data(df_subset)

 / problem type start:  0
train length of both operator selectors:  12000 12000
validation length of both operator selectors:  4000 4000
test length of both operator selectors:  2000 2000
 / problem type start:  36000
train length of both operator selectors:  12000 12000
validation length of both operator selectors:  4000 4000
test length of both operator selectors:  2000 2000
 / problem type start:  72000
train length of both operator selectors:  12000 12000
validation length of both operator selectors:  4000 4000
test length of both operator selectors:  2000 2000
 / problem type start:  108000
train length of both operator selectors:  12000 12000
validation length of both operator selectors:  4000 4000
test length of both operator selectors:  2000 2000
 / problem type start:  144000
train length of both operator selectors:  12000 12000
validation length of both operator selectors:  4000 4000
test length of both operator selectors:  2000 2000
 / problem type start:  180000
train lengt

In [74]:
training_set   = pd.concat(training_set, ignore_index=True)
validation_set = pd.concat(validation_set,   ignore_index=True)
#test_set       = pd.concat(test_set,  ignore_index=True)

In [75]:
print(training_set.shape)
print(validation_set.shape)
# print(test_set.shape)

(168000, 27)
(48000, 27)


In [82]:
training_set.to_csv('training_set_2.csv', header=True, index=False)
validation_set.to_csv('validation_set_2.csv', header=True, index=False)
#test_set.to_csv('test_set.csv', header=True, index=False)

In [50]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(training_set[["short_long_improvement"]])
training_set["y_scaled"] = scaler.transform(training_set[["short_long_improvement"]])

In [48]:
# Group data that belongs to the same state
same_state_cols = ["instance_type","tw_spread", "chosen_remove_operator", "chosen_insert_operator"]
grp_same_state = training_set.groupby(same_state_cols)
print("number of groups= ", grp_same_state.ngroups)
# for a total of 144,000 entries in training_set, 105442 groups were created, which means, there are not many entries that share the same state

number of groups=  90


In [51]:
noise_var  = grp_same_state["y_scaled"].var().mean()      # σ²_noise
print(noise_var)

0.8446677456447272


In [52]:
training_set["y_scaled"].var()

np.float64(1.0000059524163836)

## Prepare scalers for the data

In [53]:
import joblib
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler

In [54]:
train_df = pd.read_csv("training_set.csv")

In [64]:
train_df.head()


,iterations,instance_type,tw_spread,operator_selection_mechanism,number_of_vertices_to_remove,delta_last_improv,acceptance_ratio,i_last_improv,prev_remove_operator,prev_insert_operator,...,success_i_op_1,success_i_op_2,success_i_op_3,delta_cost,new_cost,chosen_remove_operator,chosen_insert_operator,current_cost,rel_delta_last_improv,short_long_improvement
0,1,0,1,1,15,0.000000,0.0,0,NaN,NaN,...,1.0,1.0,1.0,-718.948672,9548.083839,1,2,10267.032510,0.000000,-0.054251
1,2,0,1,1,5,-718.948672,1.0,0,1.0,2.0,...,1.0,1.0,1.0,-91.810910,9456.272928,1,3,9548.083839,-0.075298,-0.012359
2,3,0,1,1,5,-91.810910,1.0,0,1.0,3.0,...,1.0,1.0,1.0,-160.354919,9295.918009,4,1,9456.272928,-0.009709,-0.017553
3,4,0,1,1,5,-160.354919,1.0,0,4.0,1.0,...,1.0,1.0,1.0,0.000000,9295.918009,5,2,9295.918009,-0.017250,-0.005781
4,5,0,1,1,5,-160.354919,1.0,1,5.0,2.0,...,1.0,1.0,1.0,0.000000,9295.918009,3,1,9295.918009,-0.017250,-0.005781


In [66]:
# Prepare scaler for input features
num_pos    = ["iterations", "acceptance_ratio", "number_of_vertices_to_remove", "i_last_improv", "route_imbalance", "capacity_utilization", 'success_r_op_1', 'success_r_op_2', 'success_r_op_3', 'success_r_op_4', 'success_r_op_5', 'success_i_op_1', 'success_i_op_2','success_i_op_3']      # always positive

num_signed = ["rel_delta_last_improv"]           # can be positive or negative

cat_cols   = ["instance_type", "tw_spread", "operator_selection_mechanism",
              "prev_remove_operator", "prev_insert_operator"]             # categorical columns

# ❶  feature transformer (dense output for easy → torch)
prep = ColumnTransformer(
        transformers=[
            ("pos",    StandardScaler(), num_pos),
            ("signed", StandardScaler(), num_signed),
            ("cat",    OneHotEncoder(sparse_output=False,
                                     handle_unknown="ignore"), cat_cols),
        ])

prep.fit(train_df[num_pos + num_signed + cat_cols])
joblib.dump(prep, "../../artifacts/feature_prep.joblib")

['artifacts/feature_prep.joblib']

In [68]:
# prepare scaler for target feature
y_scaler = StandardScaler().fit(
              train_df[["short_long_improvement"]])

joblib.dump(y_scaler, "../../artifacts/y_scaler.joblib")

['artifacts/y_scaler.joblib']